# <span style="font-family:Courier New; color:#CCCCCC">**Named Entity Recognition CRF**</span>

In [1]:
import nltk
from feature_getter import Feature_getter
nltk.download('conll2002')
from nltk.corpus import conll2002
conll2002.iob_sents('esp.train') # Train, ned.train => Neerlandès
conll2002.iob_sents('esp.testa') # Dev
data = conll2002.iob_sents('esp.testb') # Test

[nltk_data] Downloading package conll2002 to
[nltk_data]     /home/jordigb/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


In [2]:
from NER_evaluation import *
entities = collect_ne(data[:5])
other = entities.copy()
other[4] = []
eval_metrics, eval_agg_ent = compute_metrics(entities, other)
eval_agg_ent

{'ORG': {'correct': 2,
  'incorrect': 0,
  'partial': 0,
  'missed': 1,
  'spurious': 0,
  'possible': 3,
  'actual': 2},
 'PER': {'correct': 2,
  'incorrect': 0,
  'partial': 0,
  'missed': 0,
  'spurious': 0,
  'possible': 2,
  'actual': 2},
 'LOC': {'correct': 2,
  'incorrect': 0,
  'partial': 0,
  'missed': 0,
  'spurious': 0,
  'possible': 2,
  'actual': 2},
 'MISC': {'correct': 3,
  'incorrect': 0,
  'partial': 0,
  'missed': 2,
  'spurious': 0,
  'possible': 5,
  'actual': 3}}

In [3]:
import nltk
train = conll2002.iob_sents('esp.train')

In [4]:
features = Feature_getter()
features.fit(train)

In [ ]:
model = nltk.tag.CRFTagger(features)